In [91]:
# reload meta_env_agent to update changes
import sys
from tqdm import tqdm
import numpy as np
import importlib
importlib.reload(sys.modules['meta_env_agent'])
from meta_env_agent import A2CAgent, DynamicBandit,  BanditEnv, Config, collect_episode_data, fit_pca_on_task_difference
from sklearn.decomposition import PCA



In [92]:
config = Config()

a2c_agent = A2CAgent(config.INPUT_SIZE, config.HIDDEN_SIZE, config.OUTPUT_SIZE,
                        lr=config.LEARNING_RATE,
                        gamma=config.GAMMA,
                        value_loss_coef=config.VALUE_LOSS_COEF,
                        entropy_coef=config.ENTROPY_COEF)

In [93]:
a2c_agent.load("a2c_agent_10000epi.pth")  # 모델 로드
a2c_agent.eval()  # 평가 모드로 설정


/Users/sskim/Documents/Research/Study/CCN_Course/meta_env_agent.py:279: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  """


A2CAgent(
  (lstm_cell): LSTMCell(2, 32)
  (action_head): Linear(in_features=32, out_features=2, bias=True)
  (value_head): Linear(in_features=32, out_features=1, bias=True)
)

In [94]:

env = DynamicBandit(training=False)


In [ ]:
NUM_TEST_EPISODES = 500 # 분석의 신뢰도를 높이기 위해 여러 에피소드 실행
all_episode_data = []

# 분석할 특정 p0 값들을 번갈아 가며 환경 생성
# p0_pairs_to_test = [[0.05, 0.95], [0.1, 0.9], [0.15, 0.85], [0.2, 0.8],
#                     [0.25, 0.75], [0.3, 0.7], [0.35, 0.65], [0.4, 0.6],
#                     [0.45, 0.55], [0.5, 0.5], [0.55, 0.45], [0.6, 0.4],
#                     [0.65, 0.35], [0.7, 0.3], [0.75, 0.25], [0.8, 0.2],
#                     [0.85, 0.15], [0.9, 0.1], [0.95, 0.05]]
p0_pairs_to_test =[[0.4 0.6],[0.6,0.4]]
print(f"\n{NUM_TEST_EPISODES}개의 테스트 에피소드에서 데이터 수집을 시작합니다...")
env = DynamicBandit(training=False)

for i in tqdm(range(NUM_TEST_EPISODES), desc="Data Collection"):
    p0_pair = p0_pairs_to_test[i % len(p0_pairs_to_test)] # p0 쌍을 번갈아 선택
    env.reset(p0_pair=p0_pair)
    print(p0_pair)
    data = collect_episode_data(a2c_agent, env,  episode_length=config.EPISODE_LENGTH)
    all_episode_data.append(data)

# 3. PCA 모델 훈련
# 모든 hidden state를 합쳐서 PCA 모델을 학습시킴
all_hidden_states = np.vstack([d['hidden_states'] for d in all_episode_data])
pca = PCA(n_components=config.PCA_COMPONENTS)
pca.fit(all_hidden_states)
print(f"\nPCA 분석 완료. 설명된 분산: {pca.explained_variance_ratio_}")



500개의 테스트 에피소드에서 데이터 수집을 시작합니다...


Data Collection:   1%|          | 3/500 [00:00<00:21, 23.37it/s]

[0.05, 0.95]
[0.1, 0.9]
[0.15, 0.85]
[0.2, 0.8]
[0.25, 0.75]


Data Collection:   2%|▏         | 9/500 [00:00<00:20, 24.37it/s]

[0.3, 0.7]
[0.35, 0.65]
[0.4, 0.6]
[0.45, 0.55]
[0.5, 0.5]


...

[0.7, 0.3]
[0.75, 0.25]
[0.8, 0.2]
[0.85, 0.15]
[0.9, 0.1]


Data Collection:  99%|█████████▉| 495/500 [00:20<00:00, 23.50it/s]

[0.95, 0.05]
[0.05, 0.95]
[0.1, 0.9]
[0.15, 0.85]
[0.2, 0.8]


Data Collection: 100%|██████████| 500/500 [00:20<00:00, 24.63it/s]

[0.25, 0.75]
[0.3, 0.7]

PCA 분석 완료. 설명된 분산: [9.9986601e-01 4.0269802e-05]


In [96]:
pca = fit_pca_on_task_difference(all_episode_data, p0_pairs_to_test)



디버깅 정보:
Task 1 (p0=[0.05, 0.95])에 대해 수집된 에피소드 수: 0
Task 2 (p0=[0.1, 0.9])에 대해 수집된 에피소드 수: 0
실제로 수집된 p0 값들 (처음 10개): ['[0.05, 0.45]', '[0.1, 0.4]', '[0.15, 0.35]', '[0.2, 0.3]', '[0.25, 0.25]', '[0.3, 0.2]', '[0.35, 0.15000000000000002]', '[0.4, 0.09999999999999998]', '[0.45, 0.04999999999999999]', '[0.5, 0.0]']


ValueError: PCA 분석에 필요한 한 개 또는 두 개 과제에 대한 데이터가 수집되지 않았습니다. 데이터 수집 과정이나 p0 값 비교 로직을 확인해주세요.